In [2]:
# %load_ext autotime
# !pip install autotime
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import os
import warnings
import pandas as pd
import numpy as np
import time
import esm
import biotite.structure.io as bsio

from proteinttt.models.esmfold import ESMFoldTTT, DEFAULT_ESMFOLD_TTT_CFG
from proteinttt.utils.structure import calculate_tm_score, lddt_score

import seaborn as sns
import matplotlib.pyplot as plt

import torch
print(f"CUDA available: {torch.cuda.is_available()}")

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 100)

base_path = Path("/scratch/project/open-32-14/pimenol1/ProteinTTT/ProteinTTT/data/bfvd")
DATASET_PATH = Path("/scratch/project/open-32-14/antonb/bfvd/bfvd_logan.fasta")
OUTPUT_PDB = base_path / Path('predicted_structures')
SUMMARY_PATH = base_path / Path('subset_1.tsv')
SAVE_PATH = base_path / Path('subset_1_2.tsv')

CORRECT_PREDICTED_PDB = Path("/scratch/project/open-32-14/antonb/bfvd/bfvd")
# os.makedirs(CDR_PATH, exist_ok=True)

# os.makedirs(base_path, exist_ok=True)

/scratch/project/open-32-14/pimenol1/miniconda3/envs/esmfold/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: False


In [1]:
len(os.listdir(OUTPUT_PDB))

NameError: name 'os' is not defined

In [4]:
# from proteinttt.utils.parsers import fetch_and_parse_sequences
# df_new = fetch_and_parse_sequences(df, CORRECT_PREDICTED_PDB, 'id')
# df.to_csv(path_or_buf=base_path / "summary_all.tsv", sep="\t", index=False)

# Dataset

In [12]:
df = pd.read_csv(SUMMARY_PATH, sep="\t")
df

,id,sequence,lenghth,pLDDT_after,pLDDT_before,tm_score_before,tm_score_after,lddt_before,lddt_after,plddt_AlphaFold,time
0,A0A811AA88,MFLNFSKSEFDTSKNEEVKYNPFAVRSESSSSSVSEFLIPDVPKKGKKGKKNTIIKPTKFKKSKPKPLTKVVIHTKFDPEKFMNVNSIIQDGKFTF...,165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A1V0SBZ5,MVKLRDGKTVYDVKDIGTVSGAERNPNGIYLDCDSQAIPDLNKLSSDVISFLEYIGTEEMEQMQSKDNEAFTKHVQDKFPEFSLNYINIFNMLLEK...,165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A5Q2W9W8,MKKYTIVKSDSKVVAGVKLYRIKANISFGNVSKGELGGYIESEKNLSQKNNAWISGDAQIWGNAQIWGNARIWGDARIWGNARISGNARIWGDAQI...,165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0A977PRX8,MNRDEYLARQAAITSERVARVRELTVQGKSLPQIAEILGVHVRTVSSDRRIAGITQSPTDAPQWSERVERVAELSRQGLSNKIIAEILHVDERTIV...,164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A0A8T8IUY0,MTNSRATRAAELDTEISRTEANVDKHTRKGQAARARLVELKRERHALNLPAEPAASVEVIRFTKKYGGNRAYHFAAIAYRLHGPNCNHKRWSVTGK...,164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
290143,A7IUJ8,MLWSWRQSITSESFPESLTMSRSRQKFMYCWTKALDSGQFLYTWKDASSMMLSPALGPPRELRMDFLVHFFTR,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290144,A0A8S5S9B4,NINKFFHIYVLYHKKFKVYTNFSILSINRLNYTFNRHFSTYFVEIIL,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290145,G4YA02,MVTAGAVHHLNGQGNSQSVATGGSTLRPFMSAANHSNIPRVVMQEPQFSHRNMPQYVRSQGLFG,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290146,H2ELU1,MAKHSSISKVERQQQRDYQNIGKELRKHVGDDKSLRFKTTDRLLAEAQGHFKMVAELSAVKGEGAVTARRKLREMRDVRYFTDPEPTSKNAWDTKQ...,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.to_csv(path_or_buf=SUMMARY_PATH, sep="\t", index=False)

In [22]:
df

,id,sequence,lenghth,pLDDT_after,pLDDT_before,tm_score_before,tm_score_after,lddt_before,lddt_after,plddt_AlphaFold,time
0,A0A8A1V5V2,MSTICGYRTIHILYIWYFTFNTVHTTINTTGITTFTVVFILYNTRHIIQNLIFTNTILTIPAEITMFSIRTPIFKFSHGRRIRPWYNLFRIQDGKG...,272,NaN,NaN,NaN,NaN,NaN,NaN,24.365400,NaN
1,A0A6J5RQS3,MACWIIAPWRYAHNSWQGIVTFTSLPVNKSLPVTRYPASVAMTLHLRSAYCMIASVVGWERLTLRKRLLPSPPLRLLPTLIVGLRYRMGTRQGALV...,262,NaN,NaN,NaN,NaN,NaN,NaN,24.614791,NaN
2,A0A1X9T5E4,MVLNVLVAFVSIGFLHTKAQIVCMPDLPGTALDSTTEPNSEMSYQYGPVALRYRYAALNTSVCTHIPTYPSVLTQLVKIASRDSHGIVIGDVMGDT...,347,NaN,NaN,NaN,NaN,NaN,NaN,24.872775,NaN
3,A7ITD6,MRYDADTFPDAITFPNISVSKVLGTTPVTFEAFPEKYAAYILPVITIEFAIALPATMFPAVKKSVSTFPGADTFPTISMAAADGNVCVTSVRKNPF...,248,NaN,NaN,NaN,NaN,NaN,NaN,24.878686,NaN
4,A0A3G5AAX1,MGQFHYFQKMSTSHESSSEFKNVMFCYNKVAPQPNSKCYNKVPPQANLKVPQQAGTTSEFENVTTRWDHKRIQKSYNKVAPQANSKMLQQSGTTRE...,154,NaN,NaN,NaN,NaN,NaN,NaN,25.188144,NaN
...,...,...,...,...,...,...,...,...,...,...,...
290143,A0A8S5URM1,MATVISEKLTINGVGSEVVLDGFDFTGNGYVEVKNASAVSIKNCRIYGIVPEAAKSFWLHVYNDIPVKLVVENCFFGNNPASGSNKMYNLIEPTAQ...,240,NaN,NaN,NaN,NaN,NaN,NaN,97.452488,NaN
290144,UPI001156C778,QARSDIEKLKEAIRDTNKAVQSVQSSIGNLIVAIKSVQDYVNKEIVPSIAR,51,NaN,NaN,NaN,NaN,NaN,NaN,97.536193,NaN
290145,A0A8S5SFL1,MKALAFDRIAVLTAKGLPDRASYIAAGDLCQIGEKTAAGLWPVTYPTKKGQKTRWVRSLKGFLVNQNHFARISYPAKGYEKATIKSSGCGVCSAII...,243,NaN,NaN,NaN,NaN,NaN,NaN,97.578114,NaN
290146,A0A6J5LRK3,MTKLYTTIGKLIFDEPDLGLKEAVEKRKANLREADLREADLIGANLIGANLYKANLIGAYLKGANLITADLEGADLREADLIGANLIGANLYKANL...,135,NaN,NaN,NaN,NaN,NaN,NaN,97.699871,NaN


In [7]:
# new_df = pd.DataFrame(columns=df.columns)
# for i, row in df.iterrows():
#     try:
#         num = row.get('id').split('_')[1]
#         if num == '1':
#             new_df = pd.concat([new_df, pd.DataFrame([row])], ignore_index=True)
#         else:
#             continue
#     except Exception as e:
#         new_df = pd.concat([new_df, pd.DataFrame([row])], ignore_index=True)

In [8]:
# rows = []
# with open(DATASET_PATH) as fh:
#     for title, seq in SimpleFastaParser(fh):
#         rows.append({"id": title, "seq": seq, "len": len(seq)})
# df = pd.DataFrame(rows)
# df.head(5)

# Model

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

base_model = esm.pretrained.esmfold_v1().eval().to(device)

ttt_cfg = DEFAULT_ESMFOLD_TTT_CFG
ttt_cfg.steps = 20
ttt_cfg.seed = 0
model = ESMFoldTTT.ttt_from_pretrained(base_model,
                                       ttt_cfg=ttt_cfg,
                                       esmfold_config=base_model.cfg).to(device)

Using cuda


In [10]:
def predict_structure(model, sequence, pdb_id, tag, out_dir=OUTPUT_PDB):
    with torch.no_grad():
        pdb_str = model.infer_pdb(sequence)

    if torch.cuda.is_available():
        torch.cuda.synchronize()

    out_path = out_dir / f"{pdb_id}{tag}.pdb"
    out_path.write_text(pdb_str)

    struct = bsio.load_structure(out_path, extra_fields=["b_factor"])
    pLDDT = float(np.asarray(struct.b_factor, dtype=float).mean())

    return pLDDT


def fold_chain(sequence, pdb_id, *, model, tag, out_dir=OUTPUT_PDB):
    model.ttt(sequence)
    pLDDT_after = predict_structure(model, sequence, pdb_id, tag='_ttt', out_dir=out_dir)
    model.ttt_reset()
    return pLDDT_after

def calculate_metrics(true_path, pred_path):
    true_struct = bsio.load_structure(true_path, extra_fields=["b_factor"])
    plddt_alpha = float(np.asarray(true_struct.b_factor, dtype=float).mean())
    
    tm_score = calculate_tm_score(pred_path, true_path)
    lddt = lddt_score(true_path, pred_path)

    return tm_score, lddt, plddt_alpha

In [11]:
num_of_low = 0
col = 'sequence'
len_col = 'lenghth'

In [14]:
for i, row in df.iterrows():
    # if pd.isna(row['pLDDT_after']):
    #     continue
    id_seq = str(row.get("id"))
    
    # tm_score_before, lddt_before, pldd_alphafold = calculate_metrics(
    #     true_path=CORRECT_PREDICTED_PDB / f"{id_seq}.pdb",
    #     pred_path=OUTPUT_PDB / f"{id_seq}.pdb"
    # )
    # tm_score_after, lddt_after, _ = calculate_metrics(
    #     true_path=CORRECT_PREDICTED_PDB / f"{id_seq}.pdb",
    #     pred_path=OUTPUT_PDB / f"{id_seq}_ttt.pdb"
    # )
    
    true_struct = bsio.load_structure(CORRECT_PREDICTED_PDB / f"{id_seq}.pdb", extra_fields=["b_factor"])
    plddt_alpha = float(np.asarray(true_struct.b_factor, dtype=float).mean())
    
    # df.at[i, 'tm_score_before'] = tm_score_before
    # df.at[i, 'lddt_before'] = lddt_before
    
    # df.at[i, 'lddt_after'] = lddt_after
    # df.at[i, 'tm_score_after'] = tm_score_after
    
    df.at[i, 'plddt_AlphaFold'] = plddt_alpha

    # print(f"tm_score before: {tm_score_before:.3f}, after: {tm_score_after:.3f}; ")

In [13]:
# df['tm_score_difference'] = df['tm_score_after'] - df['tm_score_before']

In [ ]:
start_time = time.time()

c = 0
for i, row in df.iterrows():
    c += 1
    if row[len_col] > 400 or row[len_col] < 30 or pd.notna(row['pLDDT_before']):
        continue
    if c < 20000:
        continue
    
    id = str(row.get("id"))

    if pd.isna(row[col]):
        continue
    seq = str(row[col]).strip().upper()
    start_seq_time = time.time()
    try:
        pLDDT_before = predict_structure(model, seq, id, tag="")
    except Exception as e:
        warnings.warn(f"{id}: {e}")
        pLDDT_before = np.nan
    try:
        tm_score_before, lddt_before, pldd_alphafold = calculate_metrics(
        true_path=CORRECT_PREDICTED_PDB / f"{id}.pdb",
        pred_path=OUTPUT_PDB / f"{id}.pdb"
    )
    except Exception as e:
        warnings.warn(f"{id}: {e}")
        tm_score_before = np.nan
        lddt_before = np.nan
        pldd_alphafold = np.nan

    if pLDDT_before < 70:
        try:
            pLDDT_after = fold_chain(seq, id, model=model, tag="")
        except Exception as e:
            warnings.warn(f"{id}: {e}")
            pLDDT_after = np.nan
    else: 
        pLDDT_after = np.nan
        lddt_after = np.nan
        tm_score_after = np.nan

    df.at[i, 'pLDDT_after'] = pLDDT_after
    df.at[i, 'pLDDT_before'] = pLDDT_before
    df.at[i, 'time'] = time.time() - start_seq_time
    df.at[i, 'tm_score_before'] = tm_score_before
    df.at[i, 'lddt_before'] = lddt_before

    df.at[i, 'lddt_after'] = lddt_afte
    df.at[i, 'tm_score_after'] = tm_score_after

    df.at[i, 'plddt_AlphaFold'] = pldd_alphafold
#     print(f"Processed {c} sequences. pLDDT before: {pLDDT_before:.2f}, after: {pLDDT_after:.2f}")

    if c % 50 == 0:
        df.to_csv(path_or_buf=SAVE_PATH, sep="\t", index=False)
        print(f"Total time elapsed: {time.time() - start_time:.2f} seconds for {c} sequences")

    # if c%1000 == 0:
    #     print(f"Total time elapsed: {time.time() - start_time:.2f} seconds for {c} sequences")

end_time = time.time()
print(f"Total time elapsed: {end_time - start_time:.2f} seconds for {c} sequences")

df.to_csv(SAVE_PATH, sep="\t", index=False)

Processed 21030 sequences. pLDDT before: 84.11, after: nan
Processed 21031 sequences. pLDDT before: 81.78, after: nan
Processed 21035 sequences. pLDDT before: 66.75, after: 66.75
Processed 21109 sequences. pLDDT before: 41.12, after: 54.67
Processed 21110 sequences. pLDDT before: 44.94, after: 45.84
Processed 21111 sequences. pLDDT before: 76.47, after: nan
Processed 21112 sequences. pLDDT before: 84.39, after: nan
Processed 21114 sequences. pLDDT before: 72.84, after: nan
Processed 21117 sequences. pLDDT before: 25.93, after: 28.75
Processed 21118 sequences. pLDDT before: 84.27, after: nan
Processed 21119 sequences. pLDDT before: 63.37, after: 66.96
Processed 21120 sequences. pLDDT before: 54.05, after: 55.97
Processed 21121 sequences. pLDDT before: 35.05, after: 56.34
Processed 21122 sequences. pLDDT before: 43.14, after: 47.33
Processed 21123 sequences. pLDDT before: 61.64, after: 61.84
Processed 21124 sequences. pLDDT before: 43.53, after: 48.57
Processed 21126 sequences. pLDDT bef

Processed 21264 sequences. pLDDT before: 74.34, after: nan
Processed 21265 sequences. pLDDT before: 58.50, after: 58.64
Processed 21266 sequences. pLDDT before: 62.03, after: 62.03
Processed 21267 sequences. pLDDT before: 27.39, after: 42.77
Processed 21268 sequences. pLDDT before: 81.70, after: nan
Processed 21269 sequences. pLDDT before: 82.27, after: nan
Processed 21270 sequences. pLDDT before: 52.24, after: 52.24
Processed 21271 sequences. pLDDT before: 69.10, after: 74.38
Processed 21272 sequences. pLDDT before: 71.36, after: nan
Processed 21273 sequences. pLDDT before: 41.08, after: 41.19
Processed 21274 sequences. pLDDT before: 64.96, after: 66.76
Processed 21275 sequences. pLDDT before: 88.75, after: nan
Processed 21276 sequences. pLDDT before: 84.68, after: nan
Processed 21277 sequences. pLDDT before: 43.03, after: 53.96
Processed 21278 sequences. pLDDT before: 92.13, after: nan
Processed 21279 sequences. pLDDT before: 75.70, after: nan
Processed 21280 sequences. pLDDT before:

Processed 21400 sequences. pLDDT before: 73.74, after: nan
Total time elapsed: 33348.55 seconds for 21400 sequences
Processed 21401 sequences. pLDDT before: 31.47, after: 42.82
Processed 21402 sequences. pLDDT before: 86.48, after: nan
Processed 21403 sequences. pLDDT before: 63.81, after: 66.63
Processed 21404 sequences. pLDDT before: 36.46, after: 40.43
Processed 21405 sequences. pLDDT before: 79.21, after: nan
Processed 21406 sequences. pLDDT before: 77.83, after: nan
Processed 21407 sequences. pLDDT before: 24.20, after: 30.61
Processed 21408 sequences. pLDDT before: 55.74, after: 76.98
Processed 21409 sequences. pLDDT before: 82.28, after: nan
Processed 21410 sequences. pLDDT before: 80.48, after: nan
Processed 21411 sequences. pLDDT before: 89.27, after: nan
Processed 21412 sequences. pLDDT before: 72.37, after: nan
Processed 21413 sequences. pLDDT before: 72.35, after: nan
Processed 21414 sequences. pLDDT before: 42.15, after: 48.28
Processed 21415 sequences. pLDDT before: 61.47

Processed 21535 sequences. pLDDT before: 41.65, after: 62.22
Processed 21536 sequences. pLDDT before: 76.16, after: nan
Processed 21537 sequences. pLDDT before: 75.10, after: nan
Processed 21538 sequences. pLDDT before: 76.62, after: nan
Processed 21539 sequences. pLDDT before: 85.66, after: nan
Processed 21540 sequences. pLDDT before: 35.13, after: 40.32
Processed 21541 sequences. pLDDT before: 58.72, after: 65.65
Processed 21542 sequences. pLDDT before: 88.88, after: nan
Processed 21543 sequences. pLDDT before: 83.25, after: nan
Processed 21545 sequences. pLDDT before: 87.52, after: nan
Processed 21546 sequences. pLDDT before: 44.04, after: 49.88
Processed 21547 sequences. pLDDT before: 48.87, after: 53.19
Processed 21548 sequences. pLDDT before: 89.63, after: nan
Processed 21549 sequences. pLDDT before: 89.64, after: nan
Processed 21550 sequences. pLDDT before: 41.29, after: 47.38
Total time elapsed: 46680.30 seconds for 21550 sequences
Processed 21551 sequences. pLDDT before: 69.70

In [ ]:
df.to_csv(SAVE_PATH, sep="\t", index=False)

In [ ]:
sub_df = df.query("pLDDT_before.notna() and pLDDT_after.notna()")
print(f"Entries with pLDDT before and after: {len(sub_df):,}")

sns.kdeplot(sub_df['pLDDT_before'], label='ESMFold', fill=True)
sns.kdeplot(sub_df['pLDDT_after'], label='ESMFold + ProteinTTT', fill=True)
# sns.kdeplot(sub_df['plddt_AlphaFold'], label='AlphaFold', fill=True)
plt.xlabel('pLDDT')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
sns.boxplot(data=sub_df[['pLDDT_before', 'pLDDT_after']])
sns.swarmplot(data=sub_df[['pLDDT_before', 'pLDDT_after']], color=".25")
plt.ylabel('pLDDT')
plt.xticks([0, 1], ['ESMFold', 'ESMFold + ProteinTTT'])
plt.show()  

In [ ]:
print(np.corrcoef(sub_df['pLDDT_before'], sub_df['tm_score_before'])[0, 1], np.corrcoef(sub_df['pLDDT_after'], y=sub_df['tm_score_after'])[0, 1])
sns.scatterplot(data=sub_df, x='pLDDT_before', y='tm_score_before', label='ESMFold')
sns.scatterplot(data=sub_df, x='pLDDT_after', y='tm_score_after', label='ESMFold + ProteinTTT')
plt.xlabel('pLDDT')
plt.ylabel('TM-score')
plt.legend()
plt.show()

In [ ]:
print(np.corrcoef(sub_df['pLDDT_before'], sub_df['lddt_before'])[0, 1], np.corrcoef(sub_df['pLDDT_after'], sub_df['lddt_after'])[0, 1])
sns.scatterplot(sub_df, x='pLDDT_before', y='lddt_before', label='ESMFold')
sns.scatterplot(sub_df, x='pLDDT_after', y='lddt_after', label='ESMFold + ProteinTTT')
plt.xlabel('pLDDT')
plt.ylabel('LDDT')
plt.legend()

In [ ]:
plt.figure(figsize=(6, 6))
sns.boxplot(data=sub_df[['tm_score_difference']])
sns.swarmplot(data=sub_df[['tm_score_difference']], color=".25")
plt.ylabel('tm_score_difference')
# plt.xticks([0, 1], ['tm_score_difference'])
plt.show()

In [ ]:
sub_df.pLDDT_before.describe(), sub_df.pLDDT_after.describe()

In [ ]:
sub_df